# Evolver Loop 1 - LB Feedback Analysis

## Baseline Submission Results
- CV Score: 70.6761
- LB Score: 70.6761
- Gap: 0.0000 (perfect alignment!)
- Target: 68.894234
- Gap to target: 1.78 points (2.5%)

## Key Insights from Research

### From why-not.ipynb kernel:
1. **bbox3.cpp** - Sophisticated C++ optimizer with:
   - Complex number vector coordination
   - Fluid dynamics-inspired movement
   - Hinge pivot optimization
   - Global boundary tension
   - Aggressive overlap repair cycles

2. **Lattice crystallization patterns**:
   - Blue Phase: Trees pointing up (0° ± 90°)
   - Pink Phase: Trees pointing down (180° ± 90°)
   - These interlock efficiently for large N

### From bbox3-runner kernel:
- Multi-phase optimization:
  - Phase A: 2 min runs to find promising settings
  - Phase B: 10 min runs on top candidates
  - Phase C: 20 min runs on best few
- Parameters: -n (iterations), -r (rounds)

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import os
import glob

# Tree geometry
TREE_VERTICES = [
    (0, 0.8), (-0.125, 0.5), (-0.05, 0.5), (-0.2, 0.25), (-0.1, 0.25),
    (-0.35, 0), (-0.075, 0), (-0.075, -0.2), (0.075, -0.2), (0.075, 0),
    (0.35, 0), (0.1, 0.25), (0.2, 0.25), (0.05, 0.5), (0.125, 0.5),
]

def create_tree_polygon(x, y, angle_deg):
    poly = Polygon(TREE_VERTICES)
    poly = rotate(poly, angle_deg, origin=(0, 0))
    poly = translate(poly, x, y)
    return poly

def get_bounding_box_side(polygons):
    if not polygons:
        return 0
    all_coords = []
    for poly in polygons:
        all_coords.extend(list(poly.exterior.coords))
    xs = [c[0] for c in all_coords]
    ys = [c[1] for c in all_coords]
    return max(max(xs) - min(xs), max(ys) - min(ys))

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def calculate_submission_score(csv_path):
    df = pd.read_csv(csv_path)
    df['x_val'] = df['x'].apply(parse_value)
    df['y_val'] = df['y'].apply(parse_value)
    df['deg_val'] = df['deg'].apply(parse_value)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    
    score = 0
    for n in range(1, 201):
        n_data = df[df['n'] == n]
        if len(n_data) != n:
            continue
        polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) 
                   for _, row in n_data.iterrows()]
        side = get_bounding_box_side(polygons)
        score += side**2 / n
    return score

print("Scoring function ready")

In [ ]:
# Check all available CSV files and their scores
csv_paths = [
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/chistyakov/submission_best.csv',
]

print("Checking scores of available CSV files...")
for path in csv_paths:
    if os.path.exists(path):
        try:
            score = calculate_submission_score(path)
            print(f"{os.path.basename(path)}: {score:.6f}")
        except Exception as e:
            print(f"{os.path.basename(path)}: ERROR - {e}")

In [ ]:
# Check santa25-public directory for potentially better solutions
public_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/'
if os.path.exists(public_dir):
    csv_files = glob.glob(os.path.join(public_dir, '*.csv'))
    print(f"\nFound {len(csv_files)} CSV files in santa25-public:")
    
    scores = []
    for path in csv_files:
        try:
            score = calculate_submission_score(path)
            scores.append((os.path.basename(path), score))
            print(f"  {os.path.basename(path)}: {score:.6f}")
        except Exception as e:
            print(f"  {os.path.basename(path)}: ERROR - {e}")
    
    if scores:
        scores.sort(key=lambda x: x[1])
        print(f"\nBest score: {scores[0][0]} = {scores[0][1]:.6f}")

In [ ]:
# Analyze per-N contributions to understand where improvements are possible
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(baseline_path)
df['x_val'] = df['x'].apply(parse_value)
df['y_val'] = df['y'].apply(parse_value)
df['deg_val'] = df['deg'].apply(parse_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

per_n_scores = []
for n in range(1, 201):
    n_data = df[df['n'] == n]
    if len(n_data) != n:
        continue
    polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) 
               for _, row in n_data.iterrows()]
    side = get_bounding_box_side(polygons)
    contribution = side**2 / n
    efficiency = n / (side**2)  # trees per unit area
    per_n_scores.append({'n': n, 'side': side, 'contribution': contribution, 'efficiency': efficiency})

per_n_df = pd.DataFrame(per_n_scores)
print("Per-N Analysis:")
print(f"Total score: {per_n_df['contribution'].sum():.6f}")
print(f"\nTop 10 contributors (highest score contribution):")
print(per_n_df.nlargest(10, 'contribution')[['n', 'side', 'contribution', 'efficiency']])

In [ ]:
# Analyze efficiency patterns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Score contribution by N
ax1 = axes[0, 0]
ax1.bar(per_n_df['n'], per_n_df['contribution'], alpha=0.7)
ax1.set_xlabel('N')
ax1.set_ylabel('Score Contribution (side²/N)')
ax1.set_title('Score Contribution by N')
ax1.axhline(y=per_n_df['contribution'].mean(), color='r', linestyle='--', label=f'Mean: {per_n_df["contribution"].mean():.4f}')
ax1.legend()

# Plot 2: Efficiency by N
ax2 = axes[0, 1]
ax2.scatter(per_n_df['n'], per_n_df['efficiency'], alpha=0.7, s=10)
ax2.set_xlabel('N')
ax2.set_ylabel('Efficiency (trees/unit area)')
ax2.set_title('Packing Efficiency by N')

# Plot 3: Side length by N
ax3 = axes[1, 0]
ax3.scatter(per_n_df['n'], per_n_df['side'], alpha=0.7, s=10)
ax3.set_xlabel('N')
ax3.set_ylabel('Bounding Box Side')
ax3.set_title('Bounding Box Side by N')

# Plot 4: Cumulative score
ax4 = axes[1, 1]
cumulative = per_n_df['contribution'].cumsum()
ax4.plot(per_n_df['n'], cumulative)
ax4.set_xlabel('N')
ax4.set_ylabel('Cumulative Score')
ax4.set_title('Cumulative Score by N')
ax4.axhline(y=68.894234, color='g', linestyle='--', label='Target: 68.894')
ax4.legend()

plt.tight_layout()
plt.savefig('/home/code/exploration/per_n_analysis.png', dpi=100)
plt.show()
print("Saved analysis to /home/code/exploration/per_n_analysis.png")

In [ ]:
# Identify N values with worst efficiency (most room for improvement)
print("\nN values with WORST efficiency (most room for improvement):")
worst_efficiency = per_n_df.nsmallest(20, 'efficiency')
print(worst_efficiency[['n', 'side', 'contribution', 'efficiency']])

print("\nN values with BEST efficiency:")
best_efficiency = per_n_df.nlargest(10, 'efficiency')
print(best_efficiency[['n', 'side', 'contribution', 'efficiency']])

In [ ]:
# Calculate theoretical bounds
# Tree area (approximate): The tree polygon has a specific area
tree_poly = Polygon(TREE_VERTICES)
tree_area = tree_poly.area
print(f"Single tree area: {tree_area:.6f}")

# For N trees, minimum possible bounding box side (if perfect packing with no waste)
# would be sqrt(N * tree_area)
print("\nTheoretical minimum bounds (assuming perfect packing):")
for n in [1, 10, 50, 100, 200]:
    min_side = np.sqrt(n * tree_area)
    min_contribution = min_side**2 / n
    actual_side = per_n_df[per_n_df['n'] == n]['side'].values[0]
    actual_contribution = per_n_df[per_n_df['n'] == n]['contribution'].values[0]
    waste = (actual_side**2 - n * tree_area) / (actual_side**2) * 100
    print(f"N={n:3d}: min_side={min_side:.4f}, actual={actual_side:.4f}, waste={waste:.1f}%")

In [ ]:
# Summary of findings
print("="*60)
print("SUMMARY OF FINDINGS")
print("="*60)
print(f"\nCurrent baseline score: 70.6761")
print(f"Target score: 68.894234")
print(f"Gap: 1.78 points (2.5%)")
print(f"\nKey observations:")
print(f"1. Small N values (1-10) have highest per-N contribution")
print(f"2. N=1 alone contributes {per_n_df[per_n_df['n']==1]['contribution'].values[0]:.4f}")
print(f"3. Efficiency increases with N (larger N = better packing)")
print(f"4. Worst efficiency at small N values")
print(f"\nStrategic implications:")
print(f"- Optimizing small N could yield significant gains")
print(f"- Lattice patterns for large N are already efficient")
print(f"- Need to focus on N=1-20 for biggest impact")